In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import glob

live_config_files = glob.glob('cfgs_live/*', recursive=True)
symbols = []
symbol_files = {}
for f in live_config_files:
    symbol = f.split('\\')[-1]
    symbol = symbol.replace('.json', 'USDT')
    symbols.append(symbol)
    symbol_files[symbol] = f
# symbol_files
len(symbols)

16

In [18]:
# start_files = ['start_long_only.sh', 'start_long_short.sh']
# long_symbols = []
# short_symbols = []
# for filename in start_files:
#     with open(filename, 'r') as file:
#         lines = file.read().split('\n')
#         for l in lines:
#             if 'screen -dmS ' in l:
#                 symbol = l.strip().split(" ")[-1]
#                 long_symbols.append(symbol) if filename == 'start_long_only.sh' else short_symbols.append(symbol)
#
# common_in_both = list(set(long_symbols).intersection(short_symbols))
# print(f'common_in_both : {common_in_both}')
#
# seen = set()
# long_dups = [x for x in long_symbols if x in seen or seen.add(x)]
# print(f'long_dups : {long_dups}')
# seen = set()
# short_dups = [x for x in short_symbols if x in seen or seen.add(x)]
# print(f'short_dups : {short_dups}')
#
# print(f'total: {len(symbols)}')
# print(f'long: {len(long_symbols)}')
# print(f'short: {len(short_symbols)}')
#
# if len(long_symbols) + len(short_symbols) != len(symbols):
#     print('Lengths DONT MATCH!')
# else:
#     print(f'No duplicate found')

In [19]:
import pandas as pd
config = pd.read_csv('run_config.csv')
#shorts = config[(config['short_mode'] == 'p')]
short_exposure = config.loc[config['short_mode'] == 'n', 'short_exposure'].sum()
long_exposure = config.loc[config['long_mode'] == 'n', 'long_exposure'].sum()

total = short_exposure + long_exposure
print(f'short_exposure: {short_exposure}, long_exposure:{long_exposure}, total:{total}')

short_exposure: 0, long_exposure:0.0, total:0.0


In [20]:

config = pd.read_csv('run_config.csv')
#config.loc[config['short_mode'] == 'n', 'short_exposure'] = 0.025
#config.loc[config['long_mode'] == 'n', 'long_exposure'] = 0.03
#
# short_exposure = config.loc[config['short_mode'] == 'n', 'short_exposure'].sum()
# long_exposure = config.loc[config['long_mode'] == 'n', 'long_exposure'].sum()
#
# total = short_exposure + long_exposure
# print(f'short_exposure: {short_exposure}, long_exposure:{long_exposure}, total:{total}')

#config = pd.DataFrame()

# for symbol in long_symbols:
#     config = config.append({
#         'symbol':symbol,
#         'direction':'long',
#         'long_mode':'n',
#         'short_mode':'p',
#         'long_exposure':0.035,
#         'short_exposure':0.0
#     }, ignore_index=True)
#
# for symbol in short_symbols:
#     config = config.append({
#         'symbol':symbol,
#         'direction':'long_short',
#         'long_mode':'n',
#         'short_mode':'n',
#         'long_exposure':0.035,
#         'short_exposure':0.02
#     }, ignore_index=True)
#config.loc[config['short_mode'] == 'p', 'short_mode'] = 'm'
#config.loc[config['long_mode'] == 'p', 'long_mode'] = 'm'
#config.to_csv('run_config.csv')
config

,Unnamed: 0,symbol,direction,long_mode,short_mode,long_exposure,short_exposure,account
0,2,ADA,long,gs,gs,0.10,0,binance_01
1,4,LTC,long,gs,gs,0.10,0,binance_01
2,5,ETC,long,gs,gs,0.10,0,binance_01
3,7,TRX,long,gs,gs,0.10,0,binance_01
4,8,ALGO,long,gs,gs,0.10,0,binance_01
5,10,RUNE,long,gs,gs,0.10,0,binance_01
6,13,AVAX,long,gs,gs,0.11,0,binance_01


In [21]:
# #### DCA Helper
# z_dca_long = pd.read_csv('0dca.csv')
# z_dca_long = z_dca_long[(z_dca_long['Exposure'] >= 0.04)]
#
# z_dca_long_symbols = z_dca_long['symbol'].tolist()
# z_dca_long_symbols = [w.replace('USDT','') for w in z_dca_long_symbols]
# print(z_dca_long_symbols)
# config.loc[config.symbol.isin(z_dca_long_symbols), 'long_mode'] = 'm'
# #config.to_csv('run_config.csv')
config

,Unnamed: 0,symbol,direction,long_mode,short_mode,long_exposure,short_exposure,account
0,2,ADA,long,gs,gs,0.10,0,binance_01
1,4,LTC,long,gs,gs,0.10,0,binance_01
2,5,ETC,long,gs,gs,0.10,0,binance_01
3,7,TRX,long,gs,gs,0.10,0,binance_01
4,8,ALGO,long,gs,gs,0.10,0,binance_01
5,10,RUNE,long,gs,gs,0.10,0,binance_01
6,13,AVAX,long,gs,gs,0.11,0,binance_01


In [22]:


with open('run_template.sh') as f:
    run_template = f.read()

start_template = 'screen -X -S {symbol}_binance_v5_9_3_clock kill\n' \
                 'chmod +x cfgs_runtime/{symbol}.sh\n' \
                 'screen -dmS {symbol}_binance_v5_9_3_clock ./cfgs_runtime/{symbol}.sh {symbol}'
start_commands =[]
for index, row in config.iterrows():
    dir = f'cfgs_runtime/'
    run_command = f'-sm {row["short_mode"]} -sw {row["short_exposure"]} -lm {row["long_mode"]} -lw {row["long_exposure"]} '
    full_run_command = run_template.format(command = run_command)
    #print(full_run_command)
    with open(f'{dir}/{row["symbol"]}.sh','w') as writer:
        writer.write(full_run_command)

    start_commands.append(start_template.format(symbol = row["symbol"]))

full_start_command = "\n\n".join(map(str,start_commands))
print(full_start_command)
with open(f'start.sh','w') as writer:
        writer.write(full_start_command)

screen -X -S ADA_binance_v5_9_3_clock kill
chmod +x cfgs_runtime/ADA.sh
screen -dmS ADA_binance_v5_9_3_clock ./cfgs_runtime/ADA.sh ADA

screen -X -S LTC_binance_v5_9_3_clock kill
chmod +x cfgs_runtime/LTC.sh
screen -dmS LTC_binance_v5_9_3_clock ./cfgs_runtime/LTC.sh LTC

screen -X -S ETC_binance_v5_9_3_clock kill
chmod +x cfgs_runtime/ETC.sh
screen -dmS ETC_binance_v5_9_3_clock ./cfgs_runtime/ETC.sh ETC

screen -X -S TRX_binance_v5_9_3_clock kill
chmod +x cfgs_runtime/TRX.sh
screen -dmS TRX_binance_v5_9_3_clock ./cfgs_runtime/TRX.sh TRX

screen -X -S ALGO_binance_v5_9_3_clock kill
chmod +x cfgs_runtime/ALGO.sh
screen -dmS ALGO_binance_v5_9_3_clock ./cfgs_runtime/ALGO.sh ALGO

screen -X -S RUNE_binance_v5_9_3_clock kill
chmod +x cfgs_runtime/RUNE.sh
screen -dmS RUNE_binance_v5_9_3_clock ./cfgs_runtime/RUNE.sh RUNE

screen -X -S AVAX_binance_v5_9_3_clock kill
chmod +x cfgs_runtime/AVAX.sh
screen -dmS AVAX_binance_v5_9_3_clock ./cfgs_runtime/AVAX.sh AVAX


## moved to analyze__live__result_of_current_config.ipynb